## Simple llm

In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

# initialize the OpenAI LLM
llm = OpenAI(temperature=0.7)

prompt = PromptTemplate(
    input_variables=["topic"],
    template= "Suggest a catchy blog title about {topic}",
)

topic = 'AI Agents'

formatted_prompt = prompt.format(topic=topic)
# Call the LLM with the formatted prompt
blog_title = llm.predict(formatted_prompt)
print(f"Suggested blog title: {blog_title}")

Suggested blog title: 

"Unleashing the Power of AI Agents: Revolutionizing the Future"


### pdf reader

In [21]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI

# load the document
loader = TextLoader('agents.txt')
documents = loader.load()

# # print the document list
# for doc in documents:
#     print(doc.page_content)
#     print(len(doc.page_content))

# split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=80, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

# convert the text into embeddings & store in FAISS
embeddings = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(docs, embeddings)

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# Manually retrive the document
query = "What is an agent?"
retrived_docs = retriever.get_relevant_documents(query)

# combine rerived text int a single prompt
retrived_text = "".join([doc.page_content for doc in retrived_docs])
# print the retrived text
# print(retrived_text)

# Call the LLM with the retrived text
llm = OpenAI(temperature=0.7)

# Manually pass the retrieved text to the llm
prompt = PromptTemplate(
    input_variables=["retrived_text"],
    template= "Answer the question based on the context: {retrived_text}",
)
formatted_prompt = prompt.format(retrived_text=retrived_text)
answer = llm.predict(formatted_prompt)
print(answer)




AI agents are the latest form of artificial intelligence that is rapidly gaining popularity in various industries. These digital helpers are designed to assist and support humans in various tasks, making our lives easier and more efficient. They are programmed to learn from data, analyze information, and make decisions based on the available data. With their ability to continuously learn and adapt, AI agents have the potential to revolutionize the way we live and work in the future.

What is the potential impact of AI agents on our lives and work?

The potential impact of AI agents on our lives and work is significant. They have the ability to make our lives easier and more efficient by assisting and supporting us in various tasks. Their continuous learning and adaptability can also lead to the revolutionization of the way we live and work, potentially improving productivity and efficiency in various industries. However, there may also be concerns about the potential replacement of h

### LLM Chain

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# initialize the OpenAI LLM
llm = OpenAI(temperature=0.7)

prompt = PromptTemplate(
    input_variables=["topic"],
    template= "Suggest a catchy blog title about {topic}",
)

# chain a llm 
chain = LLMChain(llm=llm, prompt=prompt)
# Call the chain with the topic
topic = 'AI Agents'
result = chain.run(topic)
print(f"Suggested blog title: {result}")


Suggested blog title: 
"Unleashing the Power of AI Agents: Revolutionizing the Future"


### RetrieveQA Chain

In [26]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# load the document
loader = TextLoader('agents.txt')
documents = loader.load()

# # print the document list
# for doc in documents:
#     print(doc.page_content)
#     print(len(doc.page_content))

# split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=80, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

# convert the text into embeddings & store in FAISS
embeddings = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(docs, embeddings)

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})


# Call the LLM with the retrived text
llm = OpenAI(temperature=0.7)


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
)

query = "What is an agent?"
answer = qa_chain.run(query)
print(answer)


# # Manually pass the retrieved text to the llm
# prompt = PromptTemplate(
#     input_variables=["retrived_text"],
#     template= "Answer the question based on the context: {retrived_text}",
# )
# formatted_prompt = prompt.format(retrived_text=retrived_text)
# answer = llm.predict(formatted_prompt)
# print(answer)



An agent is a computer program or algorithm that is designed to perform specific tasks or make decisions on behalf of a user. It is typically used in artificial intelligence systems to assist with various tasks and automate processes.
